In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

BERT extractive summarizer

In [1]:
!pip install transformers sentence-transformers torch datasets

In [2]:
from transformers import AutoModel, AutoTokenizer
import torch

model_name = "l3cube-pune/tamil-bert"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/450 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/660 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/951M [00:00<?, ?B/s]

Some weights of BertModel were not initialized from the model checkpoint at l3cube-pune/tamil-bert and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
tamil_text = """தமிழ்நாட்டின் தலைநகரம் சென்னை. இது இந்தியாவின் மிகப்பெரிய நகரங்களில் ஒன்றாகும்.
சென்னை தகவல் தொழில்நுட்பம், கல்வி, கலாசாரம், சினிமா ஆகிய துறைகளில் முக்கிய பங்கு வகிக்கிறது.
இங்கு மெரினா கடற்கரை, கபாலீஸ்வரர் கோவில், அருங்காட்சியகம் போன்றவை பிரசித்தமானவை.
சென்னை வெப்பமான வானிலையைக் கொண்டது மற்றும் வருடம் முழுவதும் பரபரப்பான நகரமாக விளங்குகிறது."""

In [7]:
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')
from nltk.tokenize import sent_tokenize

sentences = sent_tokenize(tamil_text)
print(sentences)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


['தமிழ்நாட்டின் தலைநகரம் சென்னை.', 'இது இந்தியாவின் மிகப்பெரிய நகரங்களில் ஒன்றாகும்.', 'சென்னை தகவல் தொழில்நுட்பம், கல்வி, கலாசாரம், சினிமா ஆகிய துறைகளில் முக்கிய பங்கு வகிக்கிறது.', 'இங்கு மெரினா கடற்கரை, கபாலீஸ்வரர் கோவில், அருங்காட்சியகம் போன்றவை பிரசித்தமானவை.', 'சென்னை வெப்பமான வானிலையைக் கொண்டது மற்றும் வருடம் முழுவதும் பரபரப்பான நகரமாக விளங்குகிறது.']


In [10]:
def get_embeddings(sentences):
    inputs = tokenizer(sentences, padding=True, truncation=True, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)
        embeddings = outputs.last_hidden_state.mean(dim=1)
    return embeddings

embeddings = get_embeddings(sentences)

In [12]:
from sklearn.metrics.pairwise import cosine_similarity

# Compare each sentence to the mean embedding (representing the overall document)
doc_embedding = embeddings.mean(dim=0, keepdim=True)
scores = cosine_similarity(doc_embedding, embeddings)[0]

# Rank sentences by score
ranked_sentences = [sent for _, sent in sorted(zip(scores, sentences), reverse=True)]

In [13]:
summary = " ".join(ranked_sentences[:2])
print("📝 Tamil Summary:\n", summary)

📝 Tamil Summary:
 இது இந்தியாவின் மிகப்பெரிய நகரங்களில் ஒன்றாகும். சென்னை வெப்பமான வானிலையைக் கொண்டது மற்றும் வருடம் முழுவதும் பரபரப்பான நகரமாக விளங்குகிறது.


Rogue score

In [14]:
!pip install indic-nlp-library

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 60.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.1/121.1 kB 11.6 MB/s eta 0:00:00


In [15]:
from indicnlp.tokenize import indic_tokenize

def tokenize_tamil(text):
    return " ".join(indic_tokenize.trivial_tokenize(text, lang='ta'))

In [16]:
reference_summary = """
சென்னை தமிழ்நாட்டின் தலைநகரம். இது கல்வி, கலாசாரம், சினிமா, தகவல் தொழில்நுட்ப துறைகளில் முக்கிய பங்கு வகிக்கிறது.
"""

generated_summary = """
சென்னை தகவல் தொழில்நுட்பம், கல்வி, கலாசாரம், சினிமா ஆகிய துறைகளில் முக்கிய பங்கு வகிக்கிறது.
இங்கு மெரினா கடற்கரை, கபாலீஸ்வரர் கோவில், அருங்காட்சியகம் போன்றவை பிரசித்தமானவை.
"""

ref_tok = tokenize_tamil(reference_summary)
gen_tok = tokenize_tamil(generated_summary)

print("Tokenized Reference:", ref_tok)
print("Tokenized Generated:", gen_tok)


Tokenized Reference: 
சென்னை தமிழ்நாட்டின் தலைநகரம் . இது கல்வி , கலாசாரம் , சினிமா , தகவல் தொழில்நுட்ப துறைகளில் முக்கிய பங்கு வகிக்கிறது . 

Tokenized Generated: 
சென்னை தகவல் தொழில்நுட்பம் , கல்வி , கலாசாரம் , சினிமா ஆகிய துறைகளில் முக்கிய பங்கு வகிக்கிறது . 
இங்கு மெரினா கடற்கரை , கபாலீஸ்வரர் கோவில் , அருங்காட்சியகம் போன்றவை பிரசித்தமானவை . 



mT5-small Tamil Abstractive

In [ ]:
!pip install transformers sentencepiece torch

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_name = "google/mt5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

In [ ]:
tamil_text = """
தமிழ்நாடு அரசு கல்வித் துறையில் பல புதிய முயற்சிகளை மேற்கொண்டுள்ளது.
மாணவர்கள் டிஜிட்டல் கருவிகளைப் பயன்படுத்தி கற்றல் திறன்களை மேம்படுத்துகிறார்கள்.
இந்த முயற்சி கல்வித் தரத்தை உயர்த்த உதவுகிறது."""

In [ ]:
# MT5 requires a prefix for summarization task
input_text = "summarize: " + tamil_text

# Tokenize input
inputs = tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True)

# Generate summary
summary_ids = model.generate(
    inputs.input_ids,
    max_length=80,
    min_length=20,
    length_penalty=2.0,
    num_beams=4,
    early_stopping=True
)

# Decode output
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
import re

clean_summary = re.sub(r"<extra_id_\d+>", "",summary).strip(" -\n")
print("📝 Abstractive Tamil Summary:\n", clean_summary)

📝 Abstractive Tamil Summary:
 தமிழ்நாடு அரசு கல்வித் துறையில் பல புதிய முயற்சிகளை மேற்கொண்டுள்ளது.


mT5-small Tamil Extractive

In [ ]:
!pip install transformers torch sentence-transformers nltk

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch
from nltk.tokenize import sent_tokenize
from sklearn.metrics.pairwise import cosine_similarity
import nltk
nltk.download('punkt')

# Load MT5-small as encoder-only for embeddings
model_name = "google/mt5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
tamil_text = """
தமிழ்நாடு அரசு கல்வித் துறையில் பல புதிய முயற்சிகளை மேற்கொண்டுள்ளது.
மாணவர்கள் டிஜிட்டல் கருவிகளைப் பயன்படுத்தி கற்றல் திறன்களை மேம்படுத்துகிறார்கள்.
இந்த முயற்சி கல்வித் தரத்தை உயர்த்த உதவுகிறது.
"""
sentences = sent_tokenize(tamil_text)
print("📜 Sentences:\n", sentences)

📜 Sentences:
 ['\nதமிழ்நாடு அரசு கல்வித் துறையில் பல புதிய முயற்சிகளை மேற்கொண்டுள்ளது.', 'மாணவர்கள் டிஜிட்டல் கருவிகளைப் பயன்படுத்தி கற்றல் திறன்களை மேம்படுத்துகிறார்கள்.', 'இந்த முயற்சி கல்வித் தரத்தை உயர்த்த உதவுகிறது.']


In [ ]:
def get_embeddings(sentences):
    inputs = tokenizer(sentences, padding=True, truncation=True, return_tensors="pt")
    with torch.no_grad():
        outputs = model.encoder(**inputs)  # use encoder outputs
        embeddings = outputs.last_hidden_state.mean(dim=1)  # mean pooling
    return embeddings

embeddings = get_embeddings(sentences)

In [ ]:
# Document embedding as mean of all sentence embeddings
doc_embedding = embeddings.mean(dim=0, keepdim=True)
scores = cosine_similarity(doc_embedding, embeddings)[0]

# Rank sentences by score
ranked_sentences = [sent for _, sent in sorted(zip(scores, sentences), reverse=True)]

In [ ]:
# Pick top 2 sentences
summary = " ".join(ranked_sentences[:2])
print("📝 MT5 Extractive Summary:\n", summary)

📝 MT5 Extractive Summary:
 
தமிழ்நாடு அரசு கல்வித் துறையில் பல புதிய முயற்சிகளை மேற்கொண்டுள்ளது. மாணவர்கள் டிஜிட்டல் கருவிகளைப் பயன்படுத்தி கற்றல் திறன்களை மேம்படுத்துகிறார்கள்.


Scoring - Rogue

In [ ]:
def tokenize(text):
    return text.split()

def rouge_1(ref, hyp):
    ref_tokens = set(tokenize(ref))
    hyp_tokens = set(tokenize(hyp))
    overlap = len(ref_tokens & hyp_tokens)
    precision = overlap / len(hyp_tokens)
    recall = overlap / len(ref_tokens)
    f1 = 2 * precision * recall / (precision + recall + 1e-8)
    return precision, recall, f1

def rouge_2(ref, hyp):
    def bigrams(tokens):
        return set(zip(tokens, tokens[1:]))
    ref_bigrams = bigrams(tokenize(ref))
    hyp_bigrams = bigrams(tokenize(hyp))
    overlap = len(ref_bigrams & hyp_bigrams)
    precision = overlap / (len(hyp_bigrams) + 1e-8)
    recall = overlap / (len(ref_bigrams) + 1e-8)
    f1 = 2 * precision * recall / (precision + recall + 1e-8)
    return precision, recall, f1

def rouge_l(ref, hyp):
    # Longest Common Subsequence
    ref_tokens = tokenize(ref)
    hyp_tokens = tokenize(hyp)
    m, n = len(ref_tokens), len(hyp_tokens)
    dp = [[0]*(n+1) for _ in range(m+1)]
    for i in range(m):
        for j in range(n):
            if ref_tokens[i] == hyp_tokens[j]:
                dp[i+1][j+1] = dp[i][j]+1
            else:
                dp[i+1][j+1] = max(dp[i][j+1], dp[i+1][j])
    lcs_len = dp[m][n]
    precision = lcs_len / n
    recall = lcs_len / m
    f1 = 2 * precision * recall / (precision + recall + 1e-8)
    return precision, recall, f1

In [ ]:
# Sample Tamil texts
refs = [
    "தமிழ்நாடு அரசு கல்வித் துறையில் பல புதிய முயற்சிகளை மேற்கொண்டுள்ளது. பள்ளிகளில் டிஜிட்டல் வகுப்புகள் அறிமுகம் செய்யப்பட்டுள்ளன. மாணவர்களுக்கு ஆன்லைன் வகுப்புகள் வழங்கப்படும்.",
    "சென்னை மாநகராட்சி புதிய வாகனங்கள் வாங்க திட்டமிட்டுள்ளது. போக்குவரத்து நெரிசலை குறைக்க புதிய பஸ்கள் சேவை செய்யப்படவுள்ளது.",
    "மாநில அரசால் புதிய மருத்துவமனை கட்ட திட்டம் உள்ளது. இந்த மருத்துவமனை அரசு சுகாதார சேவைகளை மேம்படுத்தும்."
]

abstractive = [
    "தமிழ்நாடு அரசு பள்ளிகளில் டிஜிட்டல் வகுப்புகளை அறிமுகப்படுத்தி, மாணவர்களுக்கு ஆன்லைன் வகுப்புகளை வழங்குகிறது.",
    "சென்னை மாநகராட்சி புதிய பஸ்கள் வாங்கி போக்குவரத்து நெரிசலை குறைக்கிறது.",
    "மாநில அரசு புதிய மருத்துவமனை கட்ட திட்டம் கொண்டு அரசு சுகாதார சேவைகளை மேம்படுத்துகிறது."
]

extractive = [
    "பள்ளிகளில் டிஜிட்டல் வகுப்புகள் அறிமுகம் செய்யப்பட்டுள்ளன.",
    "போக்குவரத்து நெரிசலை குறைக்க புதிய பஸ்கள் சேவை செய்யப்படவுள்ளது.",
    "இந்த மருத்துவமனை அரசு சுகாதார சேவைகளை மேம்படுத்தும்."
]

# --- Helper functions ---
def tokenize(text):
    return text.split()

def rouge_1(ref, hyp):
    ref_tokens = set(tokenize(ref))
    hyp_tokens = set(tokenize(hyp))
    overlap = len(ref_tokens & hyp_tokens)
    precision = overlap / len(hyp_tokens) if hyp_tokens else 0
    recall = overlap / len(ref_tokens) if ref_tokens else 0
    f1 = 2 * precision * recall / (precision + recall + 1e-8) if (precision+recall)>0 else 0
    return precision, recall, f1

def rouge_2(ref, hyp):
    def bigrams(tokens):
        return set(zip(tokens, tokens[1:]))
    ref_bigrams = bigrams(tokenize(ref))
    hyp_bigrams = bigrams(tokenize(hyp))
    overlap = len(ref_bigrams & hyp_bigrams)
    precision = overlap / len(hyp_bigrams) if hyp_bigrams else 0
    recall = overlap / len(ref_bigrams) if ref_bigrams else 0
    f1 = 2 * precision * recall / (precision + recall + 1e-8) if (precision+recall)>0 else 0
    return precision, recall, f1

def rouge_l(ref, hyp):
    ref_tokens = tokenize(ref)
    hyp_tokens = tokenize(hyp)
    m, n = len(ref_tokens), len(hyp_tokens)
    dp = [[0]*(n+1) for _ in range(m+1)]
    for i in range(m):
        for j in range(n):
            if ref_tokens[i] == hyp_tokens[j]:
                dp[i+1][j+1] = dp[i][j]+1
            else:
                dp[i+1][j+1] = max(dp[i][j+1], dp[i+1][j])
    lcs_len = dp[m][n]
    precision = lcs_len / n if n>0 else 0
    recall = lcs_len / m if m>0 else 0
    f1 = 2 * precision * recall / (precision + recall + 1e-8) if (precision+recall)>0 else 0
    return precision, recall, f1

# --- Calculate ROUGE and overall score ---
def calculate_scores(refs, hyps, summary_type):
    table = []
    for i, (r, h) in enumerate(zip(refs, hyps), 1):
        r1 = rouge_1(r,h)
        r2 = rouge_2(r,h)
        rl = rouge_l(r,h)
        overall = (r1[2] + r2[2] + rl[2])/3  # average F1
        table.append({
            "Example": i,
            "ROUGE-1 F1": round(r1[2],2),
            "ROUGE-2 F1": round(r2[2],2),
            "ROUGE-L F1": round(rl[2],2),
            "Overall ROUGE F1": round(overall,2)
        })
    return table

# --- Generate tables ---
abstractive_scores = calculate_scores(refs, abstractive, "Abstractive")
extractive_scores = calculate_scores(refs, extractive, "Extractive")

# Display tables separately
from tabulate import tabulate

print("=== Abstractive Summaries ROUGE Scores ===")
print(tabulate(abstractive_scores, headers="keys", tablefmt="grid"))

print("\n=== Extractive Summaries ROUGE Scores ===")
print(tabulate(extractive_scores, headers="keys", tablefmt="grid"))

=== Abstractive Summaries ROUGE Scores ===
+-----------+--------------+--------------+--------------+--------------------+
|   Example |   ROUGE-1 F1 |   ROUGE-2 F1 |   ROUGE-L F1 |   Overall ROUGE F1 |
+===========+==============+==============+==============+====================+
|         1 |         0.48 |         0.24 |         0.44 |               0.39 |
+-----------+--------------+--------------+--------------+--------------------+
|         2 |         0.6  |         0.42 |         0.48 |               0.5  |
+-----------+--------------+--------------+--------------+--------------------+
|         3 |         0.73 |         0.45 |         0.67 |               0.62 |
+-----------+--------------+--------------+--------------+--------------------+

=== Extractive Summaries ROUGE Scores ===
+-----------+--------------+--------------+--------------+--------------------+
|   Example |   ROUGE-1 F1 |   ROUGE-2 F1 |   ROUGE-L F1 |   Overall ROUGE F1 |
+===========+==============+======

Dataset based